In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict
#imprinting pprint for pretty printing
from pprint import pprint

In [16]:
df1 = pd.read_csv('dummy_interaction.csv')
df2 = pd.read_csv("rG-SClckUS6ydztVjFWEg.csv")
df3 = pd.read_csv("s3_bucket_links_1.csv")

In [17]:
merged_df = pd.merge(df1, df2, on='item_id', how='inner')
# Merge df1 and df2 on 'ID' column
df =  pd.merge(merged_df, df3, on='item_id', how='inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 1099
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             1100 non-null   object
 1   item_id             1100 non-null   object
 2   event_type          979 non-null    object
 3   video_duration      1100 non-null   int64 
 4   duration_watched    1100 non-null   int64 
 5   TIMESTAMP           1100 non-null   int64 
 6   sno                 1100 non-null   int64 
 7   genres              1100 non-null   object
 8   childlabel          1100 non-null   object
 9   creation_timestamp  1100 non-null   int64 
 10  Object Key          1100 non-null   object
 11  URL                 1100 non-null   object
dtypes: int64(5), object(7)
memory usage: 111.7+ KB


In [18]:
df.dropna(inplace=True)

In [19]:
df.columns

Index(['user_id', 'item_id', 'event_type', 'video_duration',
       'duration_watched', 'TIMESTAMP', 'sno', 'genres', 'childlabel',
       'creation_timestamp', 'Object Key', 'URL'],
      dtype='object')

In [20]:
movies = df[['user_id', 'item_id', 'event_type', 'genres', 'Object Key', 'childlabel']]
movies


,user_id,item_id,event_type,genres,Object Key,childlabel
0,5107502f985aa0b73854b067bc41f4b3,26cbcb970c9963d063f7d587fe7ff5a9,Liked,"Outdoors,Nature,Water,Person,Adult,Female,Land...",shortVideos/26cbcb970c9963d063f7d587fe7ff5a9.mp4,"nature,outdoors,person,human,scenery,water,wat..."
1,27b0eb67497e35b7cfa94d21f04dac6a,26cbcb970c9963d063f7d587fe7ff5a9,Liked,"Outdoors,Nature,Water,Person,Adult,Female,Land...",shortVideos/26cbcb970c9963d063f7d587fe7ff5a9.mp4,"nature,outdoors,person,human,scenery,water,wat..."
3,4d9389046a07a79e6f73941cbfe05406,26cbcb970c9963d063f7d587fe7ff5a9,Shared,"Outdoors,Nature,Water,Person,Adult,Female,Land...",shortVideos/26cbcb970c9963d063f7d587fe7ff5a9.mp4,"nature,outdoors,person,human,scenery,water,wat..."
5,4b6cd851e7edd103e4777fe920ffebe4,26cbcb970c9963d063f7d587fe7ff5a9,Liked,"Outdoors,Nature,Water,Person,Adult,Female,Land...",shortVideos/26cbcb970c9963d063f7d587fe7ff5a9.mp4,"nature,outdoors,person,human,scenery,water,wat..."
6,21131740dc09d6a532fd9d23288e79d4,26cbcb970c9963d063f7d587fe7ff5a9,Liked,"Outdoors,Nature,Water,Person,Adult,Female,Land...",shortVideos/26cbcb970c9963d063f7d587fe7ff5a9.mp4,"nature,outdoors,person,human,scenery,water,wat..."
...,...,...,...,...,...,...
1095,f468121171bac06d7500fc1f937c4b11,23070d03a2b374b078414bf73666a4df,Shared,"Person,Animal,Adult,Male,Bird,Parrot,Female,Pl...",shortVideos/23070d03a2b374b078414bf73666a4df.mp4,"adult,animal,bird,female,head,male,man,parakee..."
1096,d3118133336540047cc07627c40215fe,23070d03a2b374b078414bf73666a4df,Liked,"Person,Animal,Adult,Male,Bird,Parrot,Female,Pl...",shortVideos/23070d03a2b374b078414bf73666a4df.mp4,"adult,animal,bird,female,head,male,man,parakee..."
1097,07907c62615bb3154efd910e84b5c279,23070d03a2b374b078414bf73666a4df,Shared,"Person,Animal,Adult,Male,Bird,Parrot,Female,Pl...",shortVideos/23070d03a2b374b078414bf73666a4df.mp4,"adult,animal,bird,female,head,male,man,parakee..."
1098,de9f506fd891d7d98b0e93bcb99d464f,23070d03a2b374b078414bf73666a4df,Liked,"Person,Animal,Adult,Male,Bird,Parrot,Female,Pl...",shortVideos/23070d03a2b374b078414bf73666a4df.mp4,"adult,animal,bird,female,head,male,man,parakee..."


In [21]:
# Define a mapping for movie_id and user_id
#movie_id_map = {item_id: i for i, item_id in enumerate(movies['item_id'].unique())}
#user_id_map = {user_id: i for i, user_id in enumerate(movies['user_id'].unique())}
# event_id_map = {event_type: i for i, event_type in enumerate(df['event_type'].unique())}

#movies['item_id'] = movies['item_id'].map(movie_id_map)
#movies['user_id'] = movies['user_id'].map(user_id_map)

In [22]:

movies['object_key'] = movies['Object Key']
#movies.drop('Object Key', axis=1)

<ipython-input-22-19cf2b08516e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['object_key'] = movies['Object Key']


In [23]:
#movies['tags'] = movies['genres'] + movies['event_type']

In [24]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 979 entries, 0 to 1099
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     979 non-null    object
 1   item_id     979 non-null    object
 2   event_type  979 non-null    object
 3   genres      979 non-null    object
 4   Object Key  979 non-null    object
 5   childlabel  979 non-null    object
 6   object_key  979 non-null    object
dtypes: object(7)
memory usage: 61.2+ KB


In [25]:
new_movies = movies.drop(columns =['childlabel', 'Object Key', 'genres'])

In [12]:
new_movies.head(3)
import random
#new_movies['genres'] = random.choice(new_movies['genres'])
new_movies['genres'] = new_movies['genres'][0]

In [14]:
new_movies['genres']

0       Outdoors,Nature,Water,Person,Adult,Female,Land...
1       Outdoors,Nature,Water,Person,Adult,Female,Land...
3       Outdoors,Nature,Water,Person,Adult,Female,Land...
5       Outdoors,Nature,Water,Person,Adult,Female,Land...
6       Outdoors,Nature,Water,Person,Adult,Female,Land...
                              ...                        
1095    Outdoors,Nature,Water,Person,Adult,Female,Land...
1096    Outdoors,Nature,Water,Person,Adult,Female,Land...
1097    Outdoors,Nature,Water,Person,Adult,Female,Land...
1098    Outdoors,Nature,Water,Person,Adult,Female,Land...
1099    Outdoors,Nature,Water,Person,Adult,Female,Land...
Name: genres, Length: 979, dtype: object

In [ ]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [ ]:
new_movies['genres'] = new_movies['genres'].apply(get_list)

In [ ]:
new_movies

In [ ]:
import re

def clean_text(text):
    text = re.sub('\'', "", text)
    text = re.sub('[^a-zA-Z]', " ", text)
    text = ' '.join(text.split())
    text = text.lower()

    return text

In [ ]:
new_movies.loc[:, 'event_type']= new_movies.loc[:, 'event_type'].apply(clean_text)

In [ ]:
new_movies = pd.get_dummies(new_movies, columns=['event_type'], prefix='', prefix_sep='')

In [ ]:
new_movies


In [ ]:
from sklearn.feature_extraction.text  import CountVectorizer
new_movies.info()

In [ ]:
#cv = CountVectorizer(max_features = 1000, stop_words = 'english')

In [ ]:
#new_movies['new_event'] = new_movies['commented'] + new_movies['shared'] + new_movies['liked']

In [ ]:
#new_movies['new_event'].unique()

In [ ]:
#vector = cv.fit_transform(new_movies['tags'].values.astype('U')).toarray()

In [ ]:
#vector.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(new_movies.iloc[:,3:])

In [ ]:
similarity

In [ ]:
distance = sorted(list(enumerate(similarity[2])), reverse = True, key=lambda vector:vector[1])
for i in distance[0:5]:
    #print(i)
    print(new_movies.iloc[i[0]].user_id)

In [ ]:
def recommend(movies):
    index = new_movies[new_movies['user_id'] == movies].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse = True, key=lambda vector:vector[1])
    for i in distance[0:5]:
        print(new_movies.iloc[i[0]].item_id, new_movies.object_key)

In [ ]:
recommend('27b0eb67497e35b7cfa94d21f04dac6a')

In [ ]:
import pickle


In [ ]:
pickle.dump(new_movies, open('movies.pkl', 'wb'))


In [ ]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))

In [ ]:
pickle.load(open('movies.pkl', 'rb'))